<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Special-Variables" data-toc-modified-id="Special-Variables-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Special Variables</a></span></li><li><span><a href="#Counts-Plots" data-toc-modified-id="Counts-Plots-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Counts Plots</a></span></li><li><span><a href="#Word-Counts-Plots" data-toc-modified-id="Word-Counts-Plots-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Word Counts Plots</a></span></li></ul></div>

In [1]:
from collections import Counter
from itertools import chain
import sqlite3
import re

import holoviews as hv
from holoviews import opts
import pandas as pd
import numpy as np
from scipy import stats

# import CleanData as cd
# import BokehPlots as bp
# import CoinagePlots as cp

In [2]:
# Create connection to database
title = 'Roman_Imperial_Coinage'
fname = '../Data/' + title + '.sqlite'
conn = sqlite3.connect(fname)

# Set up holoviews backend
# hv.extension('bokeh')
renderer = hv.renderer('bokeh')


def savePlot(plot, file_name):
    """
    Save the plot at the path
    """
    #hv.renderer('bokeh').save(file_name, "../Images/{}".format(file_name), fmt='png')
    html_renderer = hv.Store.renderers['bokeh'].instance(fig='html')
    #png_renderer.save(plot, "../Images/{}".format(file_name))
    html_renderer.save(plot, "../Plots/{}".format(file_name))

# Special Variables
Edit the below variables to change how the plots are created

In [3]:
# Table names
sources = ['britishMuseum', 'americanNumismaticSociety', 'OCRE', 'allData']

# Columns to plot individually for basic counts plots
counts_columns = ["authority", 'emperor']

# Material types for counts plot (NOTE: 'None' is for all materials)
materials = [
    'total', 'silver', 'gold', 'bronze', 'lead/alloy', 'other', 'replicas'
]

# Words to show on word count plots
wanted_obverse = ['scepter', 'cuirassed', 'draped', 'laureate', 'divus']

wanted_reverse = [
    'victory', 'jupiter', 'sol', 'pax', 'cornucopia', 'emperor', 'felicitas',
    'roma', 'hercules', 'captive', 'eagle', 'trophy', 'mars', 'prince',
    'providentia', 'rudder', 'whip', 'club'
]

# Columns to plot by for the other word count plots ('Word' is for by word)
word_count_columns = ['Word', 'authority']

# Counts Plots
These stacked bar plots show the number of coins from the source selected with each bar representing the number of coins produced by each authority/emperor ordered chronologically. The material selection of "Total" returns a plot that has all of the coins regardless of the material type.

In [40]:
def countsPlot(conn):
    '''Get plot of counts of the coins by COLUMN in SOURCE 

    1st-Order Parameters
    --------------------
    conn: sqlite connection object
        Connection to sql database
        
    2nd-Order Parameters
    --------------------
    source: str
        Name of the source table
    column: str
        Column to have counts of material by typically either 'authority' or 'emperor'
    material: str
        Type of material to show. 'Total' if all

    Returns
    -------
    Holoviews plot object containing plot
    '''

    def plotBuilder(source, column, material):
        # Get data from database
        if material is 'total':
            query = '''
            SELECT
              {column},
              count({column}) AS counts
            FROM {source}
            GROUP BY {column}
            ORDER BY startDate, endDate, counts DESC;
            '''.format(
                column=column, source=source)
        else:
            query = '''
            SELECT
              {column},
              count({column}) AS counts
            FROM {source}
            WHERE material is '{material}'
            GROUP BY {column}
            ORDER BY startDate, endDate, counts DESC;
            '''.format(
                column=column, source=source, material=material)
        df = pd.read_sql(query, conn)

        # Convert dataframe into data set
        kdims = [column]
        vdims = ['counts']
        ds = hv.Dataset(df, kdims, vdims)

        # Create plot from data set
        layout = ds.to(hv.Bars, column, 'counts')
        layout.opts(
            opts.Bars(title_format=
                      'Count of {material} coins by {column} in {source} Plot'.
                      format(material=material, column=column, source=source)))

        return layout

    return plotBuilder


#countsPlot(conn)('allData', 'emperor', material=None)

In [41]:
dmap = hv.DynamicMap(
    countsPlot(conn), kdims=['Source', 'Column', 'Material']).redim.values(
        Source=sources, Column=counts_columns, Material=materials)

dmap = dmap.opts(
    opts.Bars(
        width=1500,
        height=1000,
        xrotation=90,
        tools=['hover'],
        legend_position='top_left',
    ))

In [42]:
renderer.app(dmap, show=True, new_window=True)

INFO:bokeh.server.server:Starting Bokeh server version 1.0.3 (running on Tornado 5.1.1)


INFO:tornado.access:200 GET / (::1) 322.28ms
INFO:tornado.access:200 GET /static/css/bokeh-widgets.min.css?v=6f280acb818327130a8c244ce05fd10d (::1) 5.71ms
INFO:tornado.access:200 GET /static/css/bokeh-tables.min.css?v=86c18b0b1fd30cdbc124e60cb9452e73 (::1) 1.90ms
INFO:tornado.access:200 GET /static/js/bokeh-gl.min.js?v=70e18d8dcea09947c12764bf85ec76a0 (::1) 3.04ms
INFO:tornado.access:200 GET /static/css/bokeh.min.css?v=4c253f78f16b7d5d0c9d1df8062c7f4c (::1) 3.57ms
INFO:tornado.access:200 GET /static/js/bokeh-widgets.min.js?v=68fceb4be2f3d6410ff2d7704c8b87cf (::1) 25.33ms
INFO:tornado.access:200 GET /static/js/bokeh-tables.min.js?v=02bdadb2c698bc2855a4d4a69c163821 (::1) 31.89ms
INFO:tornado.access:200 GET /static/js/bokeh.min.js?v=b343fbb4db791eda9a229febeb303a0b (::1) 52.88ms
INFO:tornado.access:101 GET /ws?bokeh-protocol-version=1.0&bokeh-session-id=w6FpzEp2JqkN8TslDaQlgvmEMznIyBBqcEcwEJXdeZDb (::1) 1.08ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.

# Word Counts Plots

In [ ]:
def wordsAnalyticsPlots(conn):
    '''Get plot of counts of the coins with WORD by COLUMN in SOURCE 

    1st-Order Parameters
    --------------------
    conn: sqlite connection object
        Connection to sql database
        
    2nd-Order Parameters
    --------------------
    source: str
        Name of the source table
    column: str
        Column to have counts of material by either 'authority' or 'emperor'
    word: str
        Type of material to sho

    Returns
    -------
    Holoviews plot object containing plot
    '''
    
    def plotBuilder(word, source):
        return